In [1]:
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, mean_absolute_error

/tmp/ipykernel_19892/4046300374.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('/home/cristina/Public/projetos/predict_price/db/dados_treino.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4094 entries, 0 to 4093
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4094 non-null   int64  
 1   latitude   4094 non-null   float64
 2   longitude  4094 non-null   float64
 3   price      4094 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 128.1 KB


In [4]:
df.head()

,id,latitude,longitude,price
0,7801,40.71842,-73.95718,299
1,14290,40.70420,-73.93560,228
2,15711,40.77065,-73.95269,250
3,16595,40.70933,-73.96792,275
4,17037,40.72162,-73.98008,230


### Divisão entre treinamento e teste

In [5]:
labels = df['price']
train_data = df[['latitude', 'longitude']]
x_train, x_test, y_train, y_test = train_test_split(train_data, labels, test_size = 0.1, random_state = 42)

obs: Antes de iniciar o treinamento, divide-se o dataframe em dados de treino e de teste para que o modelo possa avaliado quanto a sua performance.
Aqui, os dados foram divididos em 90:10, ou seja, 90% para treino e 10% para teste.

### Escolha do melhor modelo

Este é um problema de **Regressão**, pois dado utilizando-se de um conjunto de variáveis precisa-se prever um valor.
Como não há um modelo específico para cada caso, testa-se um modelo e através de um coeficiente, neste caso, a **Raiz Quadrada do Erro Médio**, é possível definir se tem uma boa performance.
O uso da Raiz Quadrada do Erro Médio foi escolhido, pois, é uma das melhores medidas de performance para problemas de regressão.

In [6]:
def best_fit(model):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    RMSE = root_mean_squared_error(y_test, y_pred)
    MAE = mean_absolute_error(y_test, y_pred)
    print("RMSE = ", RMSE)
    print("MAE = ", MAE)

Os modelos testados serão:

1. Regressão Linear

In [7]:
linear = LinearRegression()
best_fit(linear)

RMSE =  27.194493862542622
MAE =  24.230515424948994


2. Random Forest Regressor

In [8]:
rf = RandomForestRegressor()
best_fit(rf)

RMSE =  29.35042297447326
MAE =  24.798439024390245


3. KNN

In [9]:
knn = KNeighborsRegressor()
best_fit(knn)

RMSE =  28.491302909193443
MAE =  24.209756097560977


4. Gradient Boosting Regression

In [10]:
gradient = GradientBoostingRegressor()
best_fit(gradient)

RMSE =  26.825622307593818
MAE =  23.796562964492548


**Conclusão**

Quanto menor a raiz do erro quadrático médio (RMSE), melhor é a performance do modelo. Portanto,  o melhor modelo de machine learning a ser usado neste caso é o <u>Gradient Boosting Regressor</u>, cujo valor de RMSE é 26.82.

### Modelo para predição: Gradient Boosting Regressor

Gradient Boosting é um modelo de Machine Learning que funciona alinhando múltiplos modelos em sequência de modo que um modelo possa aprender com os erros dos outros e, no final, aumentar gradativamente a acurácia.
As vantagens deste algoritmo são: 
- Tem boa performance tanto para problemas de classificação quanto de regressão;
- Algoritmos de Gradient Boosting com árvore de decisão consegue lidar com dados faltantes no dataset;
- É relativamente resistente a outliers;

No entanto, suas desvantagens são:
- Demanda muito tempo, principalmente em datasets muito grandes;
- Propenso ao overfitting, em casos de modelos complexos ou de alta curva de aprendizado;
- É difícil compreensão devido a sua estrutura conjunta complexa;


- Partição em Pickle

In [17]:
model_gradient = GradientBoostingRegressor()

model_gradient.fit(x_train.values, y_train.values)

GradientBoostingRegressor()

In [18]:
with open("gradient_boosting.pkl", "wb") as f:
    pickle.dump(model_gradient, f)

In [19]:
with open("gradient_boosting.pkl", "rb") as f:
    unpickled_model_gradient = pickle.load(f)

In [20]:
pathname = './gradient_boosting.pkl'
joblib.dump(unpickled_model_gradient, pathname)

['./gradient_boosting.pkl']

In [21]:
load_model = joblib.load(pathname)

In [22]:
new_value = np.array([[40.75362, -73.98377]])
price = load_model.predict(new_value)
print('Preço: {:.2f}'.format(price[0]))

Preço: 273.49


A sugestão de preço para um apartamento com estas características:
'id': 2595,
'nome': 'Skylit Midtown Castle',
'host_id': 2845,
'host_name': 'Jennifer',
'bairro_group': 'Manhattan',
'bairro': 'Midtown',
'latitude': 40.75362,
'longitude': -73.98377,
'room_type': 'Entire home/apt',
'price': 225,
'minimo_noites': 1,
'numero_de_reviews': 45,
'ultima_review': '2019-05-21',
'reviews_por_mes': 0.38,
'calculado_host_listings_count': 2,
'disponibilidade_365': 355

é de US 273.49